<div style="display: flex; align-items: center;">
    <img src="https://github.com/nagelt/Teaching_Scripts/raw/9d9e29ecca4b04eaf7397938eacbf116d37ddc93/Images/TUBAF_Logo_blau.png" width="500" height="auto" height="auto" style="margin-right: 100px;" />
    <div>
        <p><strong>Prof. Dr. Thomas Nagel</strong></p>
        <p>Chair of Soil Mechanics and Foundation Engineering<br>Geotechnical Institute<br>Technische Universität Bergakademie Freiberg.</p>
        <p><a href="https://tu-freiberg.de/en/soilmechanics">https://tu-freiberg.de/en/soilmechanics</a></p>
    </div>
</div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import sympy as symp
from pypardiso import spsolve

#Some plot settings
%run plot_functions/plot_settings.py

# HM coupling in 1D -- liquefaction test

## Strong form

\begin{align}
    0 &= \partial_z \sigma_{zz} + \gamma_\text{r} = \partial_z (\sigma_{zz}' - p) + \gamma_\text{r} = \partial_z \left[ E_\text{s} \partial_z u_z \right] - \partial_z p + \gamma_\text{r}\\
    0 &= n\beta_p \dot{p} + \partial_z (\dot{u}_z + q_z) = n\beta_p \dot{p} + \partial_z \dot{u}_z - \partial_z \left[ \frac{k}{\mu} \left( \partial_z p - \vec{\gamma}_\text{w} \right) \right]
\end{align}

## Weak form

The displacements can have (the essential/Dirichlet) boundary conditions in the form:

$$
    u = \bar{u}\ \forall z \in \partial \Omega_\mathrm{D}% \qquad \text{and} \qquad - E_\text{s}A \frac{\partial u_z}{\partial z} = \bar{F} \ \forall z \in \partial \Omega_\mathrm{N}
$$

We now introduce a test function $\delta u$ (virtual displacement) which vanishes where the displacement is given

$$
    \delta u = 0\ \forall z \in \partial \Omega_\mathrm{D}
$$

and construct the weak form of the equilibrium conditions (using integration by parts):

\begin{align}
   0 &= \int \limits_0^H \left[\partial_z \sigma_{zz} + \gamma_\text{r} \right] \delta u\, \text{d}z
   \\
   &= \int \limits_0^H \left[\frac{\partial}{\partial z} \left(\sigma_{zz} \delta u \right) - \partial_z \delta u \ \sigma_{zz} + \gamma_\text{r} \delta u \right] \, \text{d}z
\end{align}

Integrating the first term yields the natural/Neumann boundary conditions, so that
$$
\left[ \sigma_{zz} \delta u \right]_0^H + \int \limits_0^H \gamma_\text{r} \delta u \text{d}z = \int \limits_0^H \partial_z \delta u\ (\sigma_{zz}' - p)  \, \text{d}z
$$
which is recognised as the principal of virtual work: $\delta W_\text{ext} = \delta W_\text{int}$.

The pore pressure can have (the essential/Dirichlet) boundary conditions in the form:

$$
    p = \bar{p}\ \forall z \in \partial \Omega_\mathrm{D}% \qquad \text{and} \qquad - E_\text{s}A \frac{\partial u_z}{\partial z} = \bar{F} \ \forall z \in \partial \Omega_\mathrm{N}
$$

We now introduce a test function $\delta p$ which vanishes where the pressure is given

$$
    \delta p = 0\ \forall z \in \partial \Omega_\mathrm{D}
$$

and construct the weak form of the equilibrium conditions (using integration by parts):

\begin{align}
   0 &= \int \limits_0^H \left[n\beta_p \dot{p} + \partial_z (\dot{u}_z + q_z) \right] \delta p\, \text{d}z
   \\
   &= \int \limits_0^H \left[ \partial_z (\delta p\, q_z) - q_z \partial_z \delta p + \partial_z \dot{u}_z \delta p + n\beta_p \dot{p} \delta p \right] \, \text{d}z
\end{align}

Integrating the first term yields the natural/Neumann boundary conditions, so that
$$
\left[ q_z \delta p \right]_0^H = \int \limits_0^H \left[ \partial_z \delta p\ q_z - n\beta_p \dot{p} \delta p - \delta p\, \partial_z \dot{u}_z \right] \, \text{d}z
$$

## Finite elements in 1D

We first create an element class. An element knows the number of nodes it has, their IDs in the global node vector, and the coordinates of its nodes. Linear elements have 2 nodes and 2 quadrature points, quadratic elements 3 nodes and 3 quadrature points. The natural coordinates of the element run from -1 to 1, and the quadrature points and weights are directly taken from Numpy.

In [2]:
#element class
class line_element():#local coordinates go from -1 to 1
    #takes number of nodes, global nodal coordinates, global node ids
    def __init__(self, nnodes=2, ncoords=[0.,1.], nids=[0,1]):
        self.__nnodes = nnodes
        if (len(ncoords) != self.__nnodes):
            raise Exception("Number of coordinates does not match number \
                            of nodes of element (%i vs of %i)" %(self.__nnodes,len(ncoords)))
        else:
            self.__coords = np.array(ncoords)
        
        self.__natural_coords = (self.__coords-self.__coords[0])/(self.__coords[-1]-self.__coords[0])*2. - 1.
        
        if (len(nids) != self.__nnodes):
            raise Exception("Number of node IDs does not match number \
                            of nodes of element (%i vs of %i)" %(self.__nnodes,len(nids)))
        else:
            self.__global_ids = np.array(nids)
        self.__quad_degree = self.__nnodes
        self.__quad_points, self.__quad_weights = np.polynomial.legendre.leggauss(self.__quad_degree)
                

Next, we wish to generate a one-dimensional mesh by specifying the length of a line, the number of elements into which the mesh is to be split, and the number of nodes per element.

In [3]:
def number_of_nodes(nelems,nodes_per_elem):
    return nelems*nodes_per_elem - (nelems - 1)

def generate_node_ids_u(num_iterations,max_order,p_order):
    sequence = []
    last_element = 0

    for i in range(num_iterations):
        if i == 0:
            #el_tuple = [0, 1, 2]
            el_tuple = [x for x in range(max_order)]
        elif i == 1:
            #el_tuple = [2, 5, 6]
            el_tuple = [max_order-1,max_order+p_order]
            for j in range(2,max_order):
                el_tuple.append(el_tuple[-1]+1)
        else:
            #a = last_element
            #b = a + max_order-1
            #c = b + 1
            #el_tuple = [a, b, c]
            el_tuple = [last_element,last_element + p_order]
            for j in range(2,max_order):
                el_tuple.append(el_tuple[-1]+1)
        
        sequence.append(el_tuple)
        last_element = el_tuple[-1]
    return sequence

def generate_node_ids_p(num_iterations,max_order,p_order):
    sequence = []
    last_element = max_order  # Start with the first element as 3

    for i in range(num_iterations):
        if i == 0:
            #pair = (3, 4)
            el_tuple = [x for x in range(max_order,max_order+p_order)]
        else:
            #a = last_element
            #b = a + 3
            #pair = (a, b)
            el_tuple = [last_element, last_element+max_order]
            for j in range(2,p_order):
                el_tuple.append(el_tuple[-1]+1)
        
        sequence.append(el_tuple)
        last_element = el_tuple[-1]
    return sequence

In [4]:
#use this for consequtive node numbering (global system split in [u,p])
'''
def generate_node_ids_u(num_iterations,max_order,p_order):
    sequence = []
    last_element = 0

    for i in range(num_iterations):
        el_tuple = [last_element+x for x in range(max_order)]
        sequence.append(el_tuple)
        last_element = el_tuple[-1]
    return sequence

def generate_node_ids_p(num_iterations,max_order,p_order):
    sequence = []
    last_element = number_of_nodes(num_iterations,max_order)  # Start with the first element as 3

    for i in range(num_iterations):
        el_tuple = [last_element+x for x in range(p_order)]
        sequence.append(el_tuple)
        last_element = el_tuple[-1]
    return sequence
'''

'\ndef generate_node_ids_u(num_iterations,max_order,p_order):\n    sequence = []\n    last_element = 0\n\n    for i in range(num_iterations):\n        el_tuple = [last_element+x for x in range(max_order)]\n        sequence.append(el_tuple)\n        last_element = el_tuple[-1]\n    return sequence\n\ndef generate_node_ids_p(num_iterations,max_order,p_order):\n    sequence = []\n    last_element = number_of_nodes(num_iterations,max_order)  # Start with the first element as 3\n\n    for i in range(num_iterations):\n        el_tuple = [last_element+x for x in range(p_order)]\n        sequence.append(el_tuple)\n        last_element = el_tuple[-1]\n    return sequence\n'

In [5]:
def generate_mesh(domain_length,nelems,nodes_per_elem_u,nodes_per_elem_p,eltype):
    if (eltype=='u'):
        nodes_per_elem = nodes_per_elem_u
    elif (eltype=='p'):
        nodes_per_elem = nodes_per_elem_p
    nn = number_of_nodes(nelems,nodes_per_elem)
    #coordinate vector of global nodes
    global_nodal_coordinates = np.linspace(0.,domain_length,nn)
    global_solution = np.array([0.]*nn)
    
    #generate elements
    element_vector = []
    if (eltype=='u'):
        node_ids = generate_node_ids_u(nelems,nodes_per_elem_u,nodes_per_elem_p)
    elif (eltype=='p'):
        node_ids = generate_node_ids_p(nelems,nodes_per_elem_u,nodes_per_elem_p)
    for i in range(nelems):
        node_start = (nodes_per_elem-1)*i
        node_start_id = (2*nodes_per_elem-2)*i
        element_vector.append(
            line_element(nodes_per_elem,
                         global_nodal_coordinates[node_start:node_start+nodes_per_elem],
                         node_ids[i]))
        
    return global_nodal_coordinates, element_vector, global_solution

In [6]:
def split_sol(n_els, solution,max_order,p_order):
    sol_u = np.array([])
    sol_p = np.array([])
    n_ids_u = np.unique([item for sublist in generate_node_ids_u(n_els,max_order,p_order) for item in sublist])
    n_ids_p = np.unique([item for sublist in generate_node_ids_p(n_els,max_order,p_order) for item in sublist])
    return solution[n_ids_u], solution[n_ids_p]

In [7]:
#N
def shape_function(element_order,xi):
    if (element_order == 1):
            return np.array([1])
    elif (element_order == 2): #-1,1
            return np.array([(1.-xi)/2., (1.+xi)/2.])
    elif (element_order == 3): #-1, 0, 1
            return np.array([(xi - 1.)*xi/2., (1-xi)*(1+xi), (1+xi)*xi/2.])
    elif (element_order == 4): #-1, -1/3, 1/3, 1
            return np.array([9/16*(1-xi)*(xi**2 - 1/9),
                            27/16*(xi**2-1)*(xi-1/3),
                            27/16*(1-xi**2)*(xi+1/3),
                            9/16*(xi+1)*(xi**2-1/9)])
        
#dN_dxi
def dshape_function_dxi(element_order,xi):
    if (element_order == 1):
            return np.array([0])
    elif (element_order == 2): #-1,1
        return np.array([-0.5, 0.5])  #xi only later for plotting dimensions
    elif (element_order == 3):#-1,0,1
        return np.array([xi - 0.5,-2.*xi,xi + 0.5])
    elif (element_order == 4): #-1, -1/3, 1/3, 1
            return np.array([-27*xi**2/16 + 9*xi/8 + 1/16,
                            81/16*xi**2 - 9/8 * xi - 27/16, 
                            -81/16*xi**2 - 9/8 * xi +27/16,
                            27*xi**2/16 + 9*xi/8 - 1/16])

#dz_dxi
def element_jacobian(element,xi):
    element_order = element._line_element__nnodes
    Jacobian = 0.
    Jacobian += dshape_function_dxi(element_order,xi).dot(element._line_element__coords)
    return Jacobian

#dN_dz
def grad_shape_function(element,xi):
    element_order = element._line_element__nnodes
    Jac = element_jacobian(element,xi)
    return dshape_function_dxi(element_order,xi)/Jac

## Discretization of virtual work (weak form)

The discretized system is given as

\begin{align}
    {\sigma}_{zz}(z=H)\delta_{in_\text{n}} - {\sigma}_{zz}(z=0) \delta_{i0} + \bigcup \limits_{e=1}^{n_\text{el}} \int \limits_{z_e} \gamma_\text{r} N^u_i \det J \, \text{d}\xi &= \bigcup \limits_{e=1}^{n_\text{el}} \int \limits_{z_e} \nabla N_i^u (\sigma_{zz}' - p)\det J  \, \text{d}\xi
    \\
    {q}_{z}(z=H)\delta_{in_\text{n}} - {q}_{z}(z=0) \delta_{i0} &= \bigcup \limits_{e=1}^{n_\text{el}} \int \limits_{z_e} \nabla N_i^p q_z \det J  \, \text{d}\xi - \bigcup \limits_{e=1}^{n_\text{el}} \int \limits_{z_e} N_i^p \left[ \frac{\partial_z u_z - \partial_z u_z^\text{prev}}{\Delta t} + n\beta_p\frac{p - p^\text{prev}}{\Delta t} \right]\det J  \, \text{d}\xi 
\end{align}

The latter equation was modified to yield a symmetric stiffness matrix:

\begin{align}
    {\sigma}_{zz}(z=H)\delta_{in_\text{n}} - {\sigma}_{zz}(z=0) \delta_{i0} + \bigcup \limits_{e=1}^{n_\text{el}} \int \limits_{z_e} \gamma_\text{r} N^u_i \det J \, \text{d}\xi &= \bigcup \limits_{e=1}^{n_\text{el}} \int \limits_{z_e} \nabla N_i^u (\sigma_{zz}' - p)\det J  \, \text{d}\xi
    \\
    {q}_{z}(z=H)\delta_{in_\text{n}} \Delta t - {q}_{z}(z=0) \delta_{i0} \Delta t &= \bigcup \limits_{e=1}^{n_\text{el}} \int \limits_{z_e} \nabla N_i^p q_z \Delta t\det J  \, \text{d}\xi - \bigcup \limits_{e=1}^{n_\text{el}} \int \limits_{z_e} N_i^p \left[ \Delta \partial_z u_z + n\beta_p\Delta p  \right]\det J  \, \text{d}\xi 
\end{align}

What we require now is the local assembler to calculate the stiffness matrix and the local right-hand side. Local integration is performed by Gauss quadrature:

$$
    \int \limits_{-1}^1 f(\xi)\,\text{d}\xi \approx \sum \limits_{i=1}^{n_\text{gp}} f(\xi_i) w_i 
$$

## Local assember

In [8]:
def Stiffness(z):
    return 5e6

def gamma_r(z):
    return 22.e3

def gamma_w():
    return 9.81e3

def Conductivity(z):
    return 1e-5 / 1e4 # k/mu = k_f/gamma_w

def Compressibility(z):
    return 0

In [9]:
def local_assembler_res_Jac(elem_u,elem_p,sol_u,sol_p,sol_u_prev,sol_p_prev,dt,fac=1):
    order_u = elem_u._line_element__nnodes
    order_p = elem_p._line_element__nnodes
    res_loc = np.zeros(order_u + order_p)
    K_loc = np.zeros((order_u + order_p,order_u + order_p))
    z_nodes = elem_u._line_element__coords
    for i in range(elem_u._line_element__quad_degree):
        #local integration point coordinate
        xi = elem_u._line_element__quad_points[i]
        #shape function
        N_u = shape_function(order_u,xi)
        N_p = shape_function(order_p,xi)
        #gradient of shape function
        dN_u_dX = grad_shape_function(elem_u,xi)
        dN_p_dX = grad_shape_function(elem_p,xi)
        #determinant of Jacobian
        detJ = np.abs(element_jacobian(elem_u,xi))
        #integration weight
        w = elem_u._line_element__quad_weights[i]        
        #global integration point coordinate (for spatially varying properties)
        z_glob = np.dot(N_u,z_nodes)
        #evaluation of local material/structural properties
        E = Stiffness(-np.dot(dN_u_dX,sol_u))
        k_over_mu = Conductivity(-np.dot(dN_u_dX,sol_u))
        beta = Compressibility(0)
        #evaluation of local body force contributions
        gr = 0.4 * gamma_w() #increment in body force
        gw = gamma_w()
        
        #flux quantities
        
        q_z = - k_over_mu * (np.dot(dN_p_dX,sol_p) - gw)
        #print(np.dot(dN_p_dX,sol_p), gw, gr, q_z)
        s_z = E * np.dot(dN_u_dX,sol_u) - np.dot(N_p,sol_p)
        delta_e_vol = np.dot(dN_u_dX,(sol_u - sol_u_prev))
        delta_p = np.dot(N_p,(sol_p-sol_p_prev))
               
        res_u = dN_u_dX*s_z - N_u*gr
        res_p = dN_p_dX*q_z*dt - N_p * (delta_e_vol + beta * delta_p)
        
        #assembly of local Jac
        K_loc += np.block([[np.outer(dN_u_dX,dN_u_dX)*E, -np.outer(dN_u_dX,N_p)],
                           [-np.outer(N_p,dN_u_dX), -np.outer(dN_p_dX,dN_p_dX)*k_over_mu*dt - np.outer(N_p,N_p)*beta]]) * w * detJ
        
        #assembly of local RHS
        res_loc += np.block([res_u, res_p]) * w * detJ
    return K_loc, res_loc

In [10]:
#adapted to incremental scheme
def apply_Dirichlet_bc(K_glob,b_glob,solution,node_id,value):
    c = K_glob[node_id,node_id]
    if c == 0:
        c = 1
    target = solution[node_id] - value
    
    #for i in range(len(b_glob)):
    b_glob[:] -= K_glob[:,node_id] * target
    b_glob[node_id] = c*target #no increment
    K_glob[node_id,:] = 0.
    K_glob[:,node_id] = 0.
    
    K_glob[node_id,node_id] = c

    #solution[node_id] = value
    return K_glob, b_glob

Let's assume we start from $\sigma = \sigma' = u = 0$ at $t = 0$ in an unsaturated element that we completely saturate. Thus $\Delta u |_\text{top} = 0$ and $\Delta u |_\text{bot} = \gamma_\text{w} \Delta z$.

Boundary conditions are $\Delta \sigma |_\text{top} = 0$ and $u_z |_\text{bot} = \Delta u_z |_\text{bot} = 0$. Body force loading in the integration point will be $n \gamma_\text{w}$ independent of location.

In [11]:
number_of_elements = 1
L = 1
nodes_per_element_u = 3
nodes_per_element_p = 2

nodes_u,elements_u,solution_u=generate_mesh(L,number_of_elements,nodes_per_element_u,nodes_per_element_p,'u')
nodes_p,elements_p,solution_p=generate_mesh(L,number_of_elements,nodes_per_element_u,nodes_per_element_p,'p')

In [12]:
#setting pore pressure increment
solution_p[-1] = gamma_w()*L
solution_p

array([   0., 9810.])

In [13]:
solution = np.append(solution_u,solution_p)

In [14]:
n_ids_u = np.unique([item for sublist in generate_node_ids_u(len(elements_u),elements_u[0]._line_element__nnodes,elements_p[0]._line_element__nnodes) for item in sublist])
n_ids_p = np.unique([item for sublist in generate_node_ids_p(len(elements_u),elements_u[0]._line_element__nnodes,elements_p[0]._line_element__nnodes) for item in sublist])

In [15]:
K, res = local_assembler_res_Jac(elements_u[0],elements_p[0],solution_u,solution_p,solution_u,solution_p,1)

In [16]:
K, res = apply_Dirichlet_bc(K, res, solution, n_ids_u[-1], 0)

In [17]:
for i in range(len(n_ids_p)):
    K, res = apply_Dirichlet_bc(K, res, solution, n_ids_p[i], gamma_w()*nodes_p[i])

In [18]:
du = np.linalg.solve(K,-res)
du

array([-0.0005886 , -0.00044145,  0.        , -0.        , -0.        ])

In [23]:
gamma_w()*0.6/du[0]/2

np.float64(-5000000.0)

np.float64(-0.0005886)